In [2]:
import gymnasium as gym
import time 
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

In [3]:
# Managing a Warehouse
class Warehouse:
    
    def __init__(self,max_units=15,lembda=5,k=5):
        self.lembda = lembda
        self.k = k
        self.max_units = max_units
        
    def poisson(self,k,lembda=5):
        if k>self.max_units:
            return 0
        prob = (np.power(lembda,k)*np.exp(-lembda))/math.factorial(k)
        return prob
    
    def prob_demand_exceeds(self,u,lembda=5):
        if u == 0:
            return 1
        
        tot_sum = 0
        cur_u = u
        while 1:
            cur_sum = 0
            cur_sum += tot_sum
            tot_sum += self.poisson(cur_u)
            cur_u += 1
            if abs(cur_sum - tot_sum) < 1e-9:
                break
        return tot_sum
    
    def order_cost(self,u):
        cost = 0
        if u!=0:
            cost = self.k+2*u
        return cost
        
    def holding_cost(self,u):
        return u
    
    def revenue(self,u):
        return 8*u
    
    def expected_revenue(self,u):
        ans = 0
        for j in range(u):
            ans += self.revenue(j)*self.poisson(j)
        ans += self.revenue(u)*self.prob_demand_exceeds(u)
        return ans
    
    def reward(self,s,a):
        r = self.expected_revenue(s+a) - self.order_cost(a) - self.holding_cost(s)
        return r
        
    def transition_prob(self,j,s,a):
        if j>s+a:
            return 0
        if j<=s+a and j>0:
            return self.poisson(s+a-j)
        if j==0:
            return self.prob_demand_exceeds(s+a)
        
    def make_transition_matrix(self):
        transition_matrix = {}
        for i in range(self.max_units+1):
            d = {}
            for j in range(self.max_units+1):
                l = []
                if i+j > self.max_units:
                    continue
                for k in range(self.max_units+1):
                    p = self.transition_prob(k,i,j)
                    l.append(p)
                d[j] = l
            transition_matrix[i] = d
        return transition_matrix
    
    def make_reward_matrix(self):
        reward_matrix = {}
        for i in range(self.max_units+1):
            d = []
            for j in range(self.max_units+1):
                if i+j > self.max_units:
                    continue
                d.append(self.reward(i,j))
            reward_matrix[i] = d
        return reward_matrix
    
    def make_P_dict(self):
        transition_matrix = self.make_transition_matrix()
        reward_matrix = self.make_reward_matrix()
        p = {}
        for i in range(self.max_units+1):
            d = {}
            
            for j in range(len(transition_matrix[i])):
                l = []
                
                for k in range(len(transition_matrix[i][j])):
                    x = []
                    x.append(transition_matrix[i][j][k])
                    x.append(k)
                    x.append(reward_matrix[i][j])
                    x.append(False)
                    x = tuple(x)
                    l.append(x)
                d[j] = l
            p[i] = d
        return p
    
    def val_iter(self,gamma,max_iter=1000,tolerence = 1e-6):
        P_env = self.make_P_dict()
        n_states = len(P_env)
        values = np.zeros(n_states)
        policy = np.zeros(n_states)
        iter = 0
        gap_vals = []
        max_val_state = []
        while 1:
            old_values = np.array([values[i] for i in range(n_states)])
            iter += 1
            for i in range(n_states):       
                actions_i = P_env[i]           
                max_val = 0
                best_act = 0

                for j in range(len(actions_i)):     
                    action_i_j = actions_i[j]       

                    val_j_action = 0
                    for k in range(len(action_i_j)):
                        action_i_j_k = action_i_j[k]    
                        p = action_i_j_k[0]
                        s_prime = action_i_j_k[1]
                        reward = action_i_j_k[2]
                        val_j_action += p*(reward+gamma*values[s_prime])

                    if max_val < val_j_action:
                        max_val = val_j_action
                        best_act = j

                values[i] = max_val
                policy[i] = best_act
            
            max_val_state.append(max(values))
            gap_vals.append(max(abs(old_values-values)))
            if max_iter < iter or max(abs(old_values-values)) < tolerence:
                break
        gap_vals = np.array(gap_vals)
        gap_vals = gap_vals.reshape(-1,1)
        max_val_state = np.array(max_val_state)
        max_val_state = max_val_state.reshape(-1,1)
        data = np.concatenate((gap_vals,max_val_state),axis=1)
        df = pd.DataFrame(data,columns=['gap between successive value vectors','maximum value among all states'])
        return values,policy.astype(int),iter,df
    

warehouse = Warehouse()
reward_matrix = warehouse.make_reward_matrix()
transition_matrix =  warehouse.make_transition_matrix()


print('The problem \'Managing a Warehouse\' can be modelled as a finite MDP as follows :\n')
print('The states are the number of units left at the end of the day.')
print('So there are 16 states (0 to 15) because there can be at max 15 units as per question.\n')
print('And the actions are the number of units to be ordered at the end of the day.')
print('There are 16 actions (0 to 15) indicating number of units to be ordered.\n')
print('Note : In a particular state \'s\', only those actions \'a\' can be taken which')
print('follows the constraint : s + a <= 15 as the maximum number of units is 15 (given in question)\n\n')
print('Reward and Transition formula are already given in the question.')
print('I have made a class \'Warehouse\' implementing the formulas given in the questions.\n')

print('Reward Matrix (Printed in below cells) :\n')
print('reward_matrix is a dictionary where keys (let k) are states (0 to 15) and')
print('keys are mapped to a list, where ith value of list corresponds to the reward when')
print('the action i is taken in the state k\n\n')

print('Transition Matrix (Printed in below cells) :\n')
print('transition_matrix is a dictionary of dictionaries of lists where')
print('transition_matrix[i][j] is a list (let L), kth value of L gives the ')
print('probability of reaching state k when the action j is taken at state i.\n\n\n')

print('*********************************************************************************************************')
print('*********** Now Running Value Iteration method for getting Optimal Policy and Optimal values*************\n')
print('\nNOTE : I have taken gamma = 0.8 because no value of gamma is specified in question and this is not an ')
print('episodic task, hence gamma < 1 is required for convergence.\n')
optimal_values,optimal_policy,no_of_iter,table = warehouse.val_iter(0.8,100000)
print('Optimal Values (for gamma = 0.8) :\n',optimal_values)
print('\nOptimal Policy (for gamma = 0.8) :\n',optimal_policy)
print('If value of ith (0 to 15) value of policy array is j then')
print('it means that at ith state, the policy suggests placing j orders.\n\n')
print('Table representing the successive gap in value vectors and')
print('maximum value function of a state in that iteration \n')
print(table)

The problem 'Managing a Warehouse' can be modelled as a finite MDP as follows :

The states are the number of units left at the end of the day.
So there are 16 states (0 to 15) because there can be at max 15 units as per question.

And the actions are the number of units to be ordered at the end of the day.
There are 16 actions (0 to 15) indicating number of units to be ordered.

Note : In a particular state 's', only those actions 'a' can be taken which
follows the constraint : s + a <= 15 as the maximum number of units is 15 (given in question)


Reward and Transition formula are already given in the question.
I have made a class 'Warehouse' implementing the formulas given in the questions.

Reward Matrix (Printed in below cells) :

reward_matrix is a dictionary where keys (let k) are states (0 to 15) and
keys are mapped to a list, where ith value of list corresponds to the reward when
the action i is taken in the state k


Transition Matrix (Printed in below cells) :

transition_mat

In [12]:
transition_matrix

{0: [2.333141949239525e-06,
  0.0001397813907378651,
  0.0013208616482970478,
  0.0034342402855723248,
  0.00824217668537358,
  0.01813278870782187,
  0.03626557741564375,
  0.06527803934815875,
  0.104444862957054,
  0.1462228081398756,
  0.1754673697678507,
  0.1754673697678507,
  0.14037389581428056,
  0.08422433748856833,
  0.03368973499542734,
  0.006737946999085467]}

In [15]:
reward_matrix

{0: [0,
  -0.05835057956430756,
  4.613780960907967,
  8.612117801471697,
  11.487463475521178,
  12.959070191427855,
  13.026937949191733,
  11.925023241836598,
  9.987549630825036,
  7.527851705028198,
  4.778029159906211,
  1.8831443051216468,
  -1.0776779631459021,
  -4.065974153698029,
  -7.064837237436528,
  -10.064818572300943],
 1: [6.941649420435692,
  6.613780960907967,
  10.612117801471697,
  13.487463475521178,
  14.959070191427855,
  15.026937949191733,
  13.925023241836598,
  11.987549630825036,
  9.527851705028198,
  6.778029159906211,
  3.8831443051216468,
  0.9223220368540979,
  -2.065974153698029,
  -5.064837237436528,
  -8.064818572300943],
 2: [13.613780960907967,
  12.612117801471697,
  15.487463475521178,
  16.959070191427855,
  17.026937949191733,
  15.925023241836598,
  13.987549630825036,
  11.527851705028198,
  8.778029159906211,
  5.883144305121647,
  2.922322036854098,
  -0.06597415369802917,
  -3.064837237436528,
  -6.064818572300943],
 3: [19.6121178014716

In [16]:
table

,gap between successive value vectors,maximum value among all states
0,7.411259e+01,74.112595
1,2.695410e+01,87.179267
2,1.416913e+01,94.368760
3,8.688636e+00,98.783429
4,5.627934e+00,101.641132
5,3.699405e+00,103.537676
6,2.546202e+00,104.821397
7,1.739440e+00,105.694204
8,1.182410e+00,106.287242
9,8.033704e-01,106.690167
